In [38]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.models import RecommendInput, RecommendQuery
import numpy as np
from faker import Faker

client = QdrantClient(host="localhost", port=6333)
client

In [4]:
my_collection = "first collection"
client.create_collection(
    collection_name = my_collection,
    vectors_config = models.VectorParams(size=100, distance=models.Distance.COSINE)
)

True

In [7]:
data = np.random.uniform(low=-1.0, high=1.0, size=(1_000, 100))
index = list(range(1_000))

In [9]:
data[:2, :10]

array([[-0.14127018, -0.13313706, -0.12882448, -0.79986397, -0.78780064,
        -0.1798992 , -0.45869344, -0.49252511,  0.08624379, -0.34932202],
       [-0.3450304 , -0.4528272 , -0.75618241,  0.52417484,  0.45494544,
        -0.66753981, -0.85948363,  0.21396994,  0.34051075,  0.37188729]])

In [10]:
index[-10:]

[990, 991, 992, 993, 994, 995, 996, 997, 998, 999]

In [11]:
client.upsert(
    collection_name = my_collection,
    points = models.Batch(
        ids = index,
        vectors=data.tolist()
    )
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [13]:
client.retrieve(
    collection_name=my_collection,
    ids = [10, 14, 500],
    #with_vectors=True
)

[Record(id=10, payload={}, vector=None, shard_key=None, order_value=None),
 Record(id=14, payload={}, vector=None, shard_key=None, order_value=None),
 Record(id=500, payload={}, vector=None, shard_key=None, order_value=None)]

In [14]:
fake_something = Faker()
fake_something.name(), fake_something.address()

('Brianna Wilson', '031 Lisa Row\nSouth Mark, MH 14978')

In [16]:
payload = []

for i in range(1_000): 
    payload.append(
        {
            "artist": fake_something.name(),
            "song": " ".join(fake_something.words()),
            "url_song": fake_something.url(),
            "year": fake_something.year(),
            "country": fake_something.country()
        }
    )


payload[:3]

[{'artist': 'Katherine Schultz',
  'song': 'long wind area',
  'url_song': 'https://www.howard.com/',
  'year': '2010',
  'country': 'Marshall Islands'},
 {'artist': 'Samuel Walters',
  'song': 'available factor inside',
  'url_song': 'https://walls.com/',
  'year': '2018',
  'country': 'Marshall Islands'},
 {'artist': 'Michael Vega',
  'song': 'learn similar dream',
  'url_song': 'http://washington.com/',
  'year': '2014',
  'country': 'Aruba'}]

In [17]:
client.upsert(
    collection_name = my_collection,
    points = models.Batch(
        ids=index,
        vectors=data.tolist(),
        payloads=payload
    )
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [19]:
## Semantic Search

In [20]:
liri_lira_lira_theme_song = np.random.uniform(low=-1.0, high=1.0, size=(100)).tolist()

In [22]:
client.search(
    collection_name=my_collection,
    query_vector=liri_lira_lira_theme_song,
    limit=20
)  # not mentioned in the tutorial but the search method is soon to be deprecated, hence making use of query_points in the next block

C:\Users\HP\AppData\Local\Temp\ipykernel_9412\1447653867.py:1: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  client.search(


[ScoredPoint(id=40, version=1, score=0.3161383, payload={'artist': 'Cory Ochoa', 'song': 'expert story opportunity', 'url_song': 'https://www.barnett-atkins.com/', 'year': '1982', 'country': 'Gibraltar'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=850, version=1, score=0.30021137, payload={'artist': 'Sharon Duran', 'song': 'occur wonder prevent', 'url_song': 'https://valdez.info/', 'year': '2023', 'country': 'Saint Barthelemy'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=435, version=1, score=0.2751185, payload={'artist': 'Ashley Black', 'song': 'writer state establish', 'url_song': 'https://www.bautista.biz/', 'year': '1977', 'country': 'Slovenia'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=369, version=1, score=0.2717974, payload={'artist': 'Ashley Good', 'song': 'I act official', 'url_song': 'http://thompson-goodman.net/', 'year': '1990', 'country': 'Ethiopia'}, vector=None, shard_key=None, order_value=None),
 ScoredPoi

In [29]:
client.query_points(
    collection_name=my_collection,
    query=liri_lira_lira_theme_song,
    limit=20
) 

QueryResponse(points=[ScoredPoint(id=40, version=1, score=0.3161383, payload={'artist': 'Cory Ochoa', 'song': 'expert story opportunity', 'url_song': 'https://www.barnett-atkins.com/', 'year': '1982', 'country': 'Gibraltar'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=850, version=1, score=0.30021137, payload={'artist': 'Sharon Duran', 'song': 'occur wonder prevent', 'url_song': 'https://valdez.info/', 'year': '2023', 'country': 'Saint Barthelemy'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=435, version=1, score=0.2751185, payload={'artist': 'Ashley Black', 'song': 'writer state establish', 'url_song': 'https://www.bautista.biz/', 'year': '1977', 'country': 'Slovenia'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=369, version=1, score=0.2717974, payload={'artist': 'Ashley Good', 'song': 'I act official', 'url_song': 'http://thompson-goodman.net/', 'year': '1990', 'country': 'Ethiopia'}, vector=None, shard_key=None, order_value

In [30]:
indian_songs = models.Filter(
    must=[
        models.FieldCondition(
            key="country", match=models.MatchValue(value="India")
        )
    ]
)
indian_songs

Filter(should=None, min_should=None, must=[FieldCondition(key='country', match=MatchValue(value='India'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None, is_empty=None, is_null=None)], must_not=None)

In [33]:
client.query_points(
    collection_name=my_collection,
    query=liri_lira_lira_theme_song,
    query_filter=indian_songs,
    limit=2
) 

QueryResponse(points=[ScoredPoint(id=961, version=1, score=0.11896527, payload={'artist': 'Nicole Baker', 'song': 'cold wide commercial', 'url_song': 'https://www.gillespie.com/', 'year': '1975', 'country': 'India'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=595, version=1, score=0.10060212, payload={'artist': 'Kurt Brown', 'song': 'forward key decide', 'url_song': 'https://thomas.com/', 'year': '2024', 'country': 'India'}, vector=None, shard_key=None, order_value=None)])

In [36]:
#not mentioned in the tutorial again but the recommend method is also deprecated, understand query_points needs to be used as an alternative, which means query_points is indeed much powerful

AssertionError: Unknown arguments: ['positive']

In [37]:
client.recommend(
    collection_name=my_collection,
    positive=[17],
    limit=2,
)


C:\Users\HP\AppData\Local\Temp\ipykernel_9412\2701748271.py:1: DeprecationWarning: `recommend` method is deprecated and will be removed in the future. Use `query_points` instead.
  client.recommend(


[ScoredPoint(id=416, version=1, score=0.28343883, payload={'artist': 'Paul Cortez', 'song': 'section player green', 'url_song': 'http://garcia-williams.com/', 'year': '2010', 'country': 'Australia'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=655, version=1, score=0.2760445, payload={'artist': 'Christopher Horne', 'song': 'risk surface particular', 'url_song': 'https://www.smith.com/', 'year': '2010', 'country': 'Cocos (Keeling) Islands'}, vector=None, shard_key=None, order_value=None)]

In [41]:
client.query_points(
    collection_name=my_collection,
    query=RecommendQuery(
        recommend=RecommendInput(
            positive=[171, 777],
            negative=[100,444]
        )
    ),
    
    limit=2
)

QueryResponse(points=[ScoredPoint(id=355, version=1, score=0.4077281, payload={'artist': 'Nathaniel Henderson', 'song': 'Congress government life', 'url_song': 'https://www.valdez.biz/', 'year': '2024', 'country': 'Niger'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=834, version=1, score=0.26451033, payload={'artist': 'Kristine Wright', 'song': 'join do everybody', 'url_song': 'https://www.odonnell.com/', 'year': '1995', 'country': 'Tajikistan'}, vector=None, shard_key=None, order_value=None)])